In [3]:
import pandas as pd
from google.colab import drive

In [4]:
drive.mount('/content/gdrive')
DATADIR = '/content/gdrive/MyDrive/KaggleStoreSales/'

Mounted at /content/gdrive


In [5]:
dfTransactions = pd.read_csv(DATADIR + 'transactions.csv')
dfTrain = pd.read_csv(DATADIR + 'train.csv')
dfOil = pd.read_csv(DATADIR + 'oil.csv')
dfTest = pd.read_csv(DATADIR + 'test.csv')
dfStores = pd.read_csv(DATADIR + 'stores.csv')
dfHolidayEvents = pd.read_csv(DATADIR + 'holidays_events.csv')
dfHolidayEvents.drop('transferred', inplace=True, axis=1)
dfHolidayEvents.drop('description', inplace=True, axis=1)
newColumns = dfHolidayEvents.columns.values
newColumns[1] = 'holiday_type'
dfHolidayEvents.columns = newColumns

In [6]:
#combining all the data
dfTrainMerge = dfTrain.merge(dfStores, left_on=['store_nbr'], right_on=['store_nbr'], how='inner')
dfTrainMerge = dfTrainMerge.merge(dfOil, left_on=['date'], right_on=['date'], how='left')
dfTrainMerge = dfTrainMerge.fillna(axis=0, method='bfill')
dfTrainMerge = dfTrainMerge.merge(dfTransactions, left_on=['date','store_nbr'], right_on=['date','store_nbr'], how='left')
dfTrainMerge = dfTrainMerge.fillna(0)
print(dfTrainMerge.isna().mean().round(4) * 100)
print(dfTrainMerge.info())

id              0.0
date            0.0
store_nbr       0.0
family          0.0
sales           0.0
onpromotion     0.0
city            0.0
state           0.0
type            0.0
cluster         0.0
dcoilwtico      0.0
transactions    0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   city          object 
 7   state         object 
 8   type          object 
 9   cluster       int64  
 10  dcoilwtico    float64
 11  transactions  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 297.6+ MB
None


In [7]:
dfHolidayLocal = dfHolidayEvents[dfHolidayEvents['locale'] == 'Local']
print(dfHolidayLocal)
dfTrainMergeHolidayLocal = dfTrainMerge.merge(dfHolidayLocal, left_on=['date','city'], right_on=['date','locale_name'], how='left')
dfTrainMergeHolidayLocal = dfTrainMergeHolidayLocal.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayLocal.info())

           date holiday_type locale locale_name
0    2012-03-02      Holiday  Local       Manta
2    2012-04-12      Holiday  Local      Cuenca
3    2012-04-14      Holiday  Local    Libertad
4    2012-04-21      Holiday  Local    Riobamba
5    2012-05-12      Holiday  Local        Puyo
..          ...          ...    ...         ...
339  2017-12-05   Additional  Local       Quito
340  2017-12-06      Holiday  Local       Quito
341  2017-12-08      Holiday  Local        Loja
342  2017-12-08     Transfer  Local       Quito
344  2017-12-22      Holiday  Local     Salinas

[152 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3001151
Data columns (total 15 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   city          object 
 7   state         object 
 8   type        

In [8]:
dfHolidayRegional = dfHolidayEvents[dfHolidayEvents['locale'] == 'Regional']
dfTrainMergeHolidayRegional = dfTrainMergeHolidayLocal.merge(dfHolidayRegional, left_on=['date','state'], right_on=['date','locale_name'], how='left')
dfTrainMergeHolidayRegional = dfTrainMergeHolidayRegional.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayRegional.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   store_nbr       int64  
 3   family          object 
 4   sales           float64
 5   onpromotion     int64  
 6   city            object 
 7   state           object 
 8   type            object 
 9   cluster         int64  
 10  dcoilwtico      float64
 11  transactions    float64
 12  holiday_type_x  object 
 13  locale_x        object 
 14  locale_name_x   object 
 15  holiday_type_y  object 
 16  locale_y        object 
 17  locale_name_y   object 
dtypes: float64(3), int64(4), object(11)
memory usage: 435.0+ MB
None


In [9]:
dfHolidayNational = dfHolidayEvents[dfHolidayEvents['locale'] == 'National']
dfTrainMergeHolidayNational = dfTrainMergeHolidayRegional.merge(dfHolidayNational, left_on=['date'], right_on=['date'], how='left')
dfTrainMergeHolidayNational = dfTrainMergeHolidayNational.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayNational.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3008015
Data columns (total 21 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   store_nbr       int64  
 3   family          object 
 4   sales           float64
 5   onpromotion     int64  
 6   city            object 
 7   state           object 
 8   type            object 
 9   cluster         int64  
 10  dcoilwtico      float64
 11  transactions    float64
 12  holiday_type_x  object 
 13  locale_x        object 
 14  locale_name_x   object 
 15  holiday_type_y  object 
 16  locale_y        object 
 17  locale_name_y   object 
 18  holiday_type    object 
 19  locale          object 
 20  locale_name     object 
dtypes: float64(3), int64(4), object(14)
memory usage: 503.7+ MB
None


In [10]:
dfTrainMergeHolidayNational['holiday_type'] = dfTrainMergeHolidayNational['holiday_type'].astype(str) + dfTrainMergeHolidayNational['holiday_type_x'].astype(str) + dfTrainMergeHolidayNational['holiday_type_y'].astype(str)
dfTrainMergeHolidayNational.drop('holiday_type_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('holiday_type_y', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_y', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name_y', inplace=True, axis=1)
dfTrainMergeHolidayNational = dfTrainMergeHolidayNational.fillna('NONE')
print(dfTrainMergeHolidayNational.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3008015
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   city          object 
 7   state         object 
 8   type          object 
 9   cluster       int64  
 10  dcoilwtico    float64
 11  transactions  float64
 12  holiday_type  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 320.5+ MB
None


In [19]:
#label encoding
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
train = dfTrainMergeHolidayNational
encoder.fit(train[["family", "city", 'state', 'type', 'holiday_type']])
train[["family", "city", 'state', 'type', 'holiday_type']] = encoder.transform(train[["family", "city", 'state', 'type', 'holiday_type']])

In [20]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3008015
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          object 
 2   store_nbr     int64  
 3   family        float64
 4   sales         float64
 5   onpromotion   int64  
 6   city          float64
 7   state         float64
 8   type          float64
 9   cluster       int64  
 10  dcoilwtico    float64
 11  transactions  float64
 12  holiday_type  float64
dtypes: float64(8), int64(4), object(1)
memory usage: 320.5+ MB
None


In [21]:
print(train)

              id        date  store_nbr  ...  dcoilwtico  transactions  holiday_type
0              0  2013-01-01          1  ...       93.14           0.0           7.0
1              1  2013-01-01          1  ...       93.14           0.0           7.0
2              2  2013-01-01          1  ...       93.14           0.0           7.0
3              3  2013-01-01          1  ...       93.14           0.0           7.0
4              4  2013-01-01          1  ...       93.14           0.0           7.0
...          ...         ...        ...  ...         ...           ...           ...
3008011  3000883  2017-08-15          9  ...       47.57        2155.0          15.0
3008012  3000884  2017-08-15          9  ...       47.57        2155.0          15.0
3008013  3000885  2017-08-15          9  ...       47.57        2155.0          15.0
3008014  3000886  2017-08-15          9  ...       47.57        2155.0          15.0
3008015  3000887  2017-08-15          9  ...       47.57        2

In [ ]:
#dummy variables
train = dfTrainMergeHolidayNational
train = pd.get_dummies(train, columns=["family", "city", 'state', 'type', 'holiday_type'], prefix=["family", "city",'state', 'type', 'holiday_type'])
print(train.info())

In [ ]:
train = dfTrainMergeHolidayNational

In [22]:
import datetime as dt
train['date'] = train['date'].astype('datetime64[ns]')
train['date'] = pd.to_datetime(train['date'])
train['date']=train['date'].map(dt.datetime.toordinal)

In [23]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3008015
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   date          int64  
 2   store_nbr     int64  
 3   family        float64
 4   sales         float64
 5   onpromotion   int64  
 6   city          float64
 7   state         float64
 8   type          float64
 9   cluster       int64  
 10  dcoilwtico    float64
 11  transactions  float64
 12  holiday_type  float64
dtypes: float64(8), int64(5)
memory usage: 320.5 MB
None


In [26]:
train.to_csv(DATADIR + 'trainSupplementaryLabelEncoding.csv', index=False)